In [1]:
import os
os.environ['GRAPHISTRY_USERNAME'] = input("Please enter your graphistry username")
os.environ['GRAPHISTRY_PASSWORD'] = input("Please enter your graphistry password")

In [2]:
from cognee.modules.users.methods import get_default_user

from cognee.modules.data.methods import get_datasets
from cognee.modules.data.methods.get_dataset_data import get_dataset_data
from cognee.modules.data.models import Data

from cognee.modules.pipelines.tasks.Task import Task
from cognee.tasks.documents import classify_documents, check_permissions_on_documents, extract_chunks_from_documents
from cognee.tasks.graph import extract_graph_from_code
from cognee.tasks.storage import add_data_points
from cognee.shared.SourceCodeGraph import SourceCodeGraph

from cognee.modules.pipelines import run_tasks

from cognee.shared.utils import render_graph


In [3]:
user = await get_default_user()

sys:1: SAWarning: TypeDecorator UUID() will not produce a cache key because the ``cache_ok`` attribute is not set to True.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this type object's state is safe to use in a cache key, or False to disable this warning. (Background on this warning at: https://sqlalche.me/e/20/cprf)


In [4]:
existing_datasets = await get_datasets(user.id)

datasets = {}
for dataset in existing_datasets:
    dataset_name = dataset.name.replace(".", "_").replace(" ", "_")
    data_documents: list[Data] = await get_dataset_data(dataset_id = dataset.id)
    datasets[dataset_name] = data_documents
print(datasets.keys())

dict_keys(['main_dataset'])


In [5]:
tasks = [
    Task(classify_documents),
    Task(check_permissions_on_documents, user = user, permissions = ["write"]),
    Task(extract_chunks_from_documents), # Extract text chunks based on the document type.
    Task(add_data_points, task_config = { "batch_size": 10 }),
    Task(extract_graph_from_code, graph_model = SourceCodeGraph, task_config = { "batch_size": 10 }), # Generate knowledge graphs from the document chunks.
]

In [6]:
async def run_codegraph_pipeline(tasks, data_documents):
    pipeline = run_tasks(tasks, data_documents, "code_graph_pipeline")
    results = []
    async for result in pipeline:
        results.append(result)
    return(results)

In [7]:
results = await run_codegraph_pipeline(tasks, datasets["main_dataset"])

/Users/leonluithlen/miniconda3/envs/cognee/lib/python3.11/site-packages/pydantic/main.py:1522: RuntimeWarning: fields may not start with an underscore, ignoring "_metadata"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)
Failed to load graph from file: /Users/leonluithlen/projects/cognee/cognee/.cognee_system/databases/cognee_graph.pkl 
 strptime() argument 1 must be str, not None/Users/leonluithlen/miniconda3/envs/cognee/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/leonluithlen/miniconda3/envs/cognee/lib/python3.11/site-packages/pydantic/main.py:1522: RuntimeWarning: fields may not start with an underscore, ignoring "_metadata"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


In [8]:
await render_graph(None, include_nodes = True, include_labels = True)

nodes = [(UUID('568c40c6-9e8b-51e5-af13-3fe2113ad4f6'), {'updated_at': datetime.datetime(2024, 11, 11, 11, 4, 19, 328717, tzinfo=datetime.timezone.utc), 'type': 'text', 'name': 'text_wofeceodozgyqnrcnfntpnbojhhuyrqk.txt', 'raw_data_location': '/Users/leonluithlen/projects/cognee/cognee/.data_storage/main_dataset/text_wofeceodozgyqnrcnfntpnbojhhuyrqk.txt'}), (UUID('a1c73617-3218-5e6e-8386-e023ba36aca6'), {'updated_at': datetime.datetime(2024, 11, 11, 11, 4, 19, 328717, tzinfo=datetime.timezone.utc), 'text': 'FirstEnergy (NYSE:FE) Posts Earnings Results: FirstEnergy (NYSE:FE – Get Rating) posted its earnings results on Tuesday. The utilities provider reported $0. 53 earnings per share for the quarter, topping the consensus estimate of $0. 52 by $0. 01, RTT News reports. FirstEnergy had a net margin of 10. 85% and a return on equity of 17. 17%. During the same period...\n\n If the content contained herein violates any of your rights, including those of copyright, you are requested to imme

'https://hub.graphistry.com/graph/graph.html?dataset=7fc4d34a1bb345a5b739eb1c62e04b06&type=arrow&viztoken=24a8b745-a68e-4096-93da-99016b0b9ac3&usertag=70ea3572-pygraphistry-0.33.9&splashAfter=1731323112&info=true'